The Gans Project by Balu, Marvin, Frederico and Joachim 

In this project, our E-scooter company is trying to predict  effective locations in Germany to place our scooters. For the purpose of this project we are trying to attract air travellers and  tourists to use our commodity as a means of transportation. As an experiment we are limiting ourselves to 16 of the largest air hubs in Germany which are also popular tourist destinations. 

The first task is to get the lat., long., population, country, countrycode and wiki_dataid for a list of cities in Germany and put it in a dataframe. This code is designed so that in future more cities maybe added.

In [8]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import time
#!pip install python-dotenv
#%load_ext dotenv
#%dotenv
import os


API_key = 'API_KEY'
 #gets the API key from .env
#define the list of cities
cit = ['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']
# use Englisch if possible
citd =['Baden-Baden', "Berlin",  "Bonn", 'Bremen', 'Dresden', 'Dortmund', 'Düsseldorf', 'Essen', 'Frankfurt am Main', 'Hamburg', 'Hannover', 'Leipzig', "Munich", 'Münster',  'Nuremberg',  'Stuttgart']

#strategy is to first get the wikidataid and than collect the data from geoDB; openweather has no population data or at least I coul´nt see them 

cities_id = [] # initiate an empty id list
dfList = []
def demo(cities):
    cities_id = [] # initiate an empty id list
    dfList = []
    for city in cities:
        #retrieve the wikidataId
        time.sleep(1)
        url1 = 'https://en.wikipedia.org/wiki/{}'.format(city) #go to the wiki site of the city
        citem = requests.get(url1, 'html.parser') # get the html
        if BeautifulSoup(citem.content) != None:
            soup = BeautifulSoup(citem.content)# soup the content
        if soup.find('li', {'id':'t-wikibase'}).find('a')['href'] != None:
            wikidata_link = soup.find('li', {'id':'t-wikibase'}).find('a')['href'] #find the link that contains the wikidataid e.g. London https://www.wikidata.org/wiki/Q84
        #wl.append(wikidata_link)
        # \d+ is a regular expression and means one digit or more, the wiki data id consist of a Q followed by severaldigits
        #for group() in re see: https://www.tutorialspoint.com/What-is-the-groups-method-in-regular-expressions-in-Python
        city_id = re.search('Q\d+', wikidata_link).group()
        cities_id.append(city_id)
        #use the wikidataId to retrieve infrormation from geoDB
        url2 = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{}".format(city_id)
        headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
        }    
        response = requests.request("GET", url2, headers=headers)# gets a json-like string fromg geoDB containing the necessary informations
        print(response.json())
        cit_dic = {}#make a dictionary to retrieve the information
        cit_dic['City'] = response.json()['data']['name']
        cit_dic['Country'] = response.json()['data']['country']
        cit_dic['CountryCode'] = response.json()['data']['countryCode']
        cit_dic['WikiDataId'] = response.json()['data']['wikiDataId']
        cit_dic['Latitude'] = round(response.json()['data']['latitude'], 4)
        cit_dic['Longitude'] = round(response.json()['data']['longitude'], 4)
        cit_dic['Population'] = response.json()['data']['population']
        cit_dic['Timezone'] = response.json()['data']['timezone']
        
        dfList.append(cit_dic) #put it in a list
        df_demo = pd.DataFrame(dfList) # transform the list to df
        #print(dfList)
    return df_demo
demo(citd) #run the whole thing - good luck!

{'data': {'id': 25706, 'wikiDataId': 'Q4100', 'type': 'CITY', 'city': 'Baden-Baden', 'name': 'Baden-Baden', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Baden-Württemberg', 'regionCode': 'BW', 'elevationMeters': 181, 'latitude': 48.761944444, 'longitude': 8.240833333, 'population': 55449, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 3453309, 'wikiDataId': 'Q64', 'type': 'CITY', 'city': 'Berlin', 'name': 'Berlin', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Berlin', 'regionCode': 'BE', 'elevationMeters': None, 'latitude': 52.516666666, 'longitude': 13.383333333, 'population': 3664088, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 3449715, 'wikiDataId': 'Q586', 'type': 'CITY', 'city': 'Bonn', 'name': 'Bonn', 'country': 'Germany', 'countryCode': 'DE', 'region': 'North Rhine-Westphalia', 'regionCode': 'NW', 'elevationMeters': None, 'latitude': 50.733888888

,City,Country,CountryCode,WikiDataId,Latitude,Longitude,Population,Timezone
0,Baden-Baden,Germany,DE,Q4100,48.7619,8.2408,55449,Europe__Berlin
1,Berlin,Germany,DE,Q64,52.5167,13.3833,3664088,Europe__Berlin
2,Bonn,Germany,DE,Q586,50.7339,7.0997,330579,Europe__Berlin
3,Bremen,Germany,DE,Q24879,53.0769,8.8089,566573,Europe__Berlin
4,Dresden,Germany,DE,Q1731,51.0833,13.7666,554907,Europe__Berlin
5,Dortmund,Germany,DE,Q1295,51.5139,7.4653,587696,Europe__Berlin
6,Düsseldorf,Germany,DE,Q1718,51.2311,6.7724,645923,Europe__Berlin
7,Essen,Germany,DE,Q2066,51.4508,7.0131,582415,Europe__Berlin
8,Frankfurt am Main,Germany,DE,Q1794,50.1136,8.6797,764104,Europe__Berlin
9,Hamburg,Germany,DE,Q1055,53.5500,10.0000,1852478,Europe__Berlin


Focussing on these 15 cities, we are collecting the weather forecast. This DF is subject to change as the weather information is updated as needed.

In [10]:
from datetime import datetime
import pytz
#!pip install python-dotenv
#%load_ext dotenv
#%dotenv
import os

In [11]:
#cit = ['Berlin', 'London', 'Paris', 'Rome', 'Wien','Tallinn', 'Budapest', 'Warschau', 'Prague', 'Lisbon', 'Madrid', 'Amsterdam', 'Zagreb', 'Sofia', 'Athens']# use Englisch if possible
#citd =['Berlin', 'Munich', 'Frankfurt', 'Kiel', 'Rostock', 'Leipzig', 'Cologne', 'Dresden', 'Erfurt', 'Jena', 'Weimar', 'Essen', 'Stuttgart', 'Mainz', 'Hannover', 'Hamburg']#alternatively only German cities
def weather(cities):
  API_key = API #gets key from the env file

  tz = pytz.timezone('Europe/Berlin')# timezone needs to be imported and translated for the european cities (Europe_Berlin to Europe/Berlin), see cell below
  now = datetime.now().astimezone(tz)#for the timestamp time of sampling

  a = {'City': [], 'Country': [], 'ForecastTime': [], 'Outlook': [], 'Temperature': [], 'Clouds': [], 'Rain': [], 'Snow': [], 'WindSpeed': [], 'DateInformation': []} #usefull form of a dic, gives the right form when change to df, see cell below 

  for city in cities: #runs a loop over the cities
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json() # the first 'priciple' key for this json is 'list', which is not good practice in my op but must be used to loop over it
    # print(json['city']['name'])
    # print(type(json))

    for i in json['list']:# informations will be added to the dic a according to the json structure (see weimar cell above), if-else for rain does not work (is there or not but python cannot know that it could be there), therefore try except - appears to work but I´m not sure if clean
      #print(i)  
      a['City'].append(json['city']['name'])
      a['Country'].append(json['city']['country'])
      a['ForecastTime'].append(i['dt_txt'])
      a['Outlook'].append(i['weather'][0]['description'])
      a['Temperature'].append(i['main']['temp'])
      a['Clouds'].append(i['clouds']['all'])
      try:
          a['Rain'].append(i['rain']['3h'])
      except:
          a['Rain'].append('0')
      try:
          a['Snow'].append(i['snow']['3h'])
      except:
          a['Snow'].append('0')
      a['WindSpeed'].append(i['wind']['speed'])
      a['DateInformation'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
    b=pd.DataFrame(a) #turns a to datafram , see cell below
    #print(b)
  return b
weather(cit)


,City,Country,ForecastTime,Outlook,Temperature,Clouds,Rain,Snow,WindSpeed,DateInformation
0,Baden-Baden,DE,2022-06-12 12:00:00,clear sky,25.93,0,0,0,3.11,12/06/2022 11:14:57
1,Baden-Baden,DE,2022-06-12 15:00:00,clear sky,26.90,3,0,0,2.56,12/06/2022 11:14:57
2,Baden-Baden,DE,2022-06-12 18:00:00,light rain,22.87,27,0.38,0,2.02,12/06/2022 11:14:57
3,Baden-Baden,DE,2022-06-12 21:00:00,light rain,16.68,42,1.56,0,0.43,12/06/2022 11:14:57
4,Baden-Baden,DE,2022-06-13 00:00:00,light rain,15.41,62,0.94,0,1.88,12/06/2022 11:14:57
...,...,...,...,...,...,...,...,...,...,...
635,Stuttgart,DE,2022-06-16 21:00:00,broken clouds,19.19,56,0,0,0.63,12/06/2022 11:14:57
636,Stuttgart,DE,2022-06-17 00:00:00,broken clouds,17.05,76,0,0,0.32,12/06/2022 11:14:57
637,Stuttgart,DE,2022-06-17 03:00:00,overcast clouds,15.48,86,0,0,0.95,12/06/2022 11:14:57
638,Stuttgart,DE,2022-06-17 06:00:00,overcast clouds,19.47,91,0,0,1.18,12/06/2022 11:14:57


For the cities in our database we now get the details of the airports. 

In [22]:
ac = ['EDSB', 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']
airport = []
for a in ac:
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{a}"

    querystring = {"withRunways":"true"}

    headers = {
        "X-RapidAPI-Key": "API_KE",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    airport.append(response.json())

    

In [32]:
air = pd.json_normalize(airport)

air.drop(columns = ['fullName', 'runways', 'continent.code', 'urls.wikipedia', 'urls.googleMaps', 'urls.flightRadar', 'urls.twitter', 'location.lat', 'location.lon'], inplace = True)
air.columns = ['icao', 'iata', 'name', 'city', 'time_zone', 'country_code', 'country', 'continent', 'website']
air

,icao,iata,name,city,time_zone,country_code,country,continent,website
0,EDSB,FKB,Karlsruhe,Baden-Baden,Europe/Berlin,DE,Germany,Europe,http://www.baden-airpark.de/startseite.html
1,EDDT,TXL,-Tegel,Berlin,Europe/Berlin,DE,Germany,Europe,http://www.berlin-airport.de/en/travellers-txl...
2,EDDB,BER,Brandenburg,Berlin,Europe/Berlin,DE,Germany,Europe,https://ber.berlin-airport.de/
3,EDDK,CGN,Bonn,Cologne,Europe/Berlin,DE,Germany,Europe,http://www.koeln-bonn-airport.de/en/
4,EDDW,BRE,Bremen,Bremen,Europe/Berlin,DE,Germany,Europe,http://www.bremen-airport.com/
5,EDDC,DRS,Dresden,Dresden,Europe/Berlin,DE,Germany,Europe,https://www.dresden-airport.de
6,EDLW,DTM,Dortmund,Dortmund,Europe/Berlin,DE,Germany,Europe,NaN
7,EDDL,DUS,Düsseldorf,Duesseldorf,Europe/Berlin,DE,Germany,Europe,https://www.dus.com/en
8,EDDF,FRA,Frankfurt-am-Main,Frankfurt-am-Main,Europe/Berlin,DE,Germany,Europe,http://www.frankfurt-airport.de/
9,EDDH,HAM,Hamburg,Hamburg,Europe/Berlin,DE,Germany,Europe,https://www.hamburg-airport.de/en/


For all the airports in our list we can have the  details of the arriving flights. The most important detail being the flight times. The list may also optionally include the details of the flights origin as in future we can have a statistical analysis of the passengers who may use our scooters. This is also a changing DF and with queries we can find out the most and least busiest times of the day. This code is not ideal since it does not show the destination airport. Need to edit the codes.

In [103]:
flight_details = []
cities = ['EDSB', 'EDDT', 'EDDB', 'EDDK', 'EDDW', 'EDDC', 'EDLW', 'EDDL', 'EDDF', 'EDDH', 'EDDV', 'EDDP', 'EDDM', 'EDDG', 'EDDN', 'EDDS']
for city in cities:
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{city}/2022-06-08T06:00/2022-06-08T17:00"

    querystring = {"direction":"Arrival","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

    headers = {
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
        "X-RapidAPI-Key": "API_KEY"
    }

    responses = requests.request("GET", url, headers=headers, params=querystring)
    responses.json()

    flights = pd.json_normalize(responses.json()["arrivals"])
    #flight_data = flights.drop(columns = ['codeshareStatus', 'isCargo', 'movement.airport.icao','movement.terminal' 'movement.airport.iata', 'movement.quality', 'aircraft.model', 'aircraft.reg', 'aircraft.modeS', 'movement.runwayTimeLocal', 'movement.runwayTimeUtc', 'movement.runway', 'callSign'])
    flight = pd.DataFrame(flights)
    flight_details.append(flight)
    


In [106]:
fli = pd.concat(flight_details)
fli.drop(columns = ['callSign', 'codeshareStatus', 'isCargo', 'movement.airport.icao', 'movement.airport.iata', 'movement.scheduledTimeUtc', 'aircraft.reg','movement.actualTimeLocal','movement.actualTimeUtc', 'movement.runway', 'movement.runwayTimeLocal', 'movement.runwayTimeUtc', 'movement.terminal', 'movement.gate', 'movement.baggageBelt', 'movement.checkInDesk', 'movement.quality', 'aircraft.modeS'], inplace = True) 
fli.columns = ['flight_number', 'status', 'origin_airport', 'scheduled_time', 'aircraft', 'airlines']
fli['airport'] = city
fli

,flight_number,status,origin_airport,scheduled_time,aircraft,airlines,airport
0,GFD 41,Approaching,Unknown,2022-06-08 11:49+02:00,Bombardier Learjet 35,GFD,EDDS
1,FR 9772,Unknown,London,2022-06-08 08:50+02:00,Airbus A320,Ryanair,EDDS
2,W6 6617,Unknown,Tirana,2022-06-08 13:30+02:00,Airbus A320 NEO,Wizz Air,EDDS
3,FR 6117,Unknown,Sofia,2022-06-08 08:10+02:00,Boeing 737-800,Ryanair,EDDS
4,FR 9211,Approaching,Girona,2022-06-08 15:50+02:00,Boeing 737-800,Ryanair,EDDS
...,...,...,...,...,...,...,...
75,TK 1701,Arrived,Istanbul,2022-06-08 10:00+02:00,Airbus A321-200 (Sharklets),Turkish,EDDS
76,TK 1703,Arrived,Istanbul,2022-06-08 14:25+02:00,Airbus A321-200 (Sharklets),Turkish,EDDS
77,FH 321,Arrived,Antalya,2022-06-08 11:25+02:00,Airbus A320,Freebird,EDDS
78,EW 2735,Arrived,Kraków,2022-06-08 14:00+02:00,Airbus A319,Eurowings,EDDS


After completing all the DFs, they are exported to an already created SQL Schema. Note: No tables were created in MySQL workbench all tables are created from here. Once created go to sql to create Primary and Foreign Keys. Also the flights DF is not exported as the city column is missing and I have ran out of API calls.

Please feel free to alter the codes in anyway that seem fit.

In [111]:
import mysql.connector
import sqlalchemy
# define connection details
cnx = mysql.connector.connect(
    user='root',
    password='!Kochumol20', #type your root password here
    host='127.0.0.1', # to connect to your local instance
    database='gans' #type the name of the database you want to use here
)
 
# connect to database
cursor = cnx.cursor()
schema="gans"
host="127.0.0.1"
user="root"
password="password"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [116]:
weather(cit).to_sql('weather', 
              if_exists='append', 
              con=con, 
              index=False)
air.to_sql('airports', 
              if_exists='append', 
              con=con, 
              index=False)
demo(citd).to_sql('cities', 
              if_exists='append', 
              con=con, 
              index=False)

{'data': {'id': 25706, 'wikiDataId': 'Q4100', 'type': 'CITY', 'city': 'Baden-Baden', 'name': 'Baden-Baden', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Baden-Württemberg', 'regionCode': 'BW', 'elevationMeters': 181, 'latitude': 48.761944444, 'longitude': 8.240833333, 'population': 55449, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 3453309, 'wikiDataId': 'Q64', 'type': 'CITY', 'city': 'Berlin', 'name': 'Berlin', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Berlin', 'regionCode': 'BE', 'elevationMeters': None, 'latitude': 52.516666666, 'longitude': 13.383333333, 'population': 3664088, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 3449715, 'wikiDataId': 'Q586', 'type': 'CITY', 'city': 'Bonn', 'name': 'Bonn', 'country': 'Germany', 'countryCode': 'DE', 'region': 'North Rhine-Westphalia', 'regionCode': 'NW', 'elevationMeters': None, 'latitude': 50.733888888